# Análisis de resultados GA + ACO
Este notebook genera visualizaciones a partir de los archivos en `output/` y `datos/`.

Gráficos incluidos (una celda por insight):
- Convergencia del GA (mejor costo por generación).
- Convergencia del ACO (mejor costo por iteración para la última corrida).
- Barras del mejor cromosoma vs baseline y bandas 90%-110%.
- Layout espacial hexagonal coloreado por especie.
- Mapa de calor por nodo de la competencia acumulada con vecinos.
- Matrices especie–especie: adyacencias y competencia ponderada.
- Resumen de ejecución (tiempo, generaciones, early-stopping, etc.).

Nota: las celdas son tolerantes a que falten archivos; en ese caso reportan el faltante.

In [ ]:
# Setup e imports
from pathlib import Path
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

ROOT = Path(__file__).resolve().parents[0]
DATOS = ROOT / 'datos'
OUTPUT = ROOT / 'output'
FIGS = OUTPUT / 'figs'
FIGS.mkdir(parents=True, exist_ok=True)

def exists(p: Path) -> bool:
    return p.exists() and p.is_file()

# Paleta para 10 especies
PALETTE = [
    '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
    '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'
]

BASELINE = np.array([42, 196, 42, 42, 49, 38, 73, 64, 86, 26])
LB = np.floor(0.9 * BASELINE).astype(int)
UB = np.ceil(1.1 * BASELINE).astype(int)
SUMA_OBJ = 658

# Paths de archivos de salida esperados
P_MEJOR_CROM = OUTPUT / 'mejor_cromosoma_genetico.npy'
P_HIST_GA = OUTPUT / 'historial_costos_ga.npy'
P_RESUMEN = OUTPUT / 'resumen_ga.json'
P_ASIG = OUTPUT / 'mejor_asignacion_hormigas.npy'
P_HIST_ACO = OUTPUT / 'historial_costos_aco.npy'

# Datos auxiliares
P_GRAFO = DATOS / 'grafo_hexagonal.json'
P_COMP = DATOS / 'matriz_competencia.npy'

print('Directorio raíz:', ROOT)
print('Figuras se guardarán en:', FIGS)

## Insight 1: Convergencia del GA
Curva del mejor costo por generación (menor es mejor). Fuente: `output/historial_costos_ga.npy`.

In [ ]:
if exists(P_HIST_GA):
    hist = np.load(P_HIST_GA)
    plt.figure(figsize=(7,4))
    plt.plot(np.arange(1, len(hist)+1), hist, marker='o', linewidth=1.5)
    plt.xlabel('Generación')
    plt.ylabel('Mejor costo')
    plt.title('GA: Mejor costo por generación')
    plt.grid(True, alpha=0.3)
    out = FIGS / 'ga_convergencia.png'
    plt.savefig(out, dpi=150, bbox_inches='tight')
    plt.show()
    print('Guardado:', out)
else:
    print('Falta archivo:', P_HIST_GA)

## Insight 2: Convergencia del ACO
Curva del mejor costo por iteración para la última corrida de ACO. Fuente: `output/historial_costos_aco.npy`.

In [ ]:
if exists(P_HIST_ACO):
    hist_aco = np.load(P_HIST_ACO)
    plt.figure(figsize=(7,4))
    plt.plot(np.arange(1, len(hist_aco)+1), hist_aco, color='tab:orange', marker='o', linewidth=1.5)
    plt.xlabel('Iteración ACO')
    plt.ylabel('Mejor costo')
    plt.title('ACO: Mejor costo por iteración')
    plt.grid(True, alpha=0.3)
    out = FIGS / 'aco_convergencia.png'
    plt.savefig(out, dpi=150, bbox_inches='tight')
    plt.show()
    print('Guardado:', out)
else:
    print('Falta archivo:', P_HIST_ACO)

## Insight 3: Mejor cromosoma vs baseline y bandas 90%–110%
Barras por especie; línea del baseline y bandas permitidas. Fuente: `output/mejor_cromosoma_genetico.npy`.

In [ ]:
if exists(P_MEJOR_CROM):
    crom = np.load(P_MEJOR_CROM)
    especies = np.arange(1, len(crom)+1)
    plt.figure(figsize=(9,4.8))
    # bandas
    plt.fill_between(especies, LB, UB, color='lightgray', alpha=0.5, label='Banda 90%-110%')
    # baseline
    plt.plot(especies, BASELINE, '--', color='black', label='Baseline')
    # barras
    plt.bar(especies, crom, color='tab:blue', alpha=0.8, label='Mejor cromosoma')
    plt.xticks(especies, [f'E{i}' for i in especies])
    plt.xlabel('Especie')
    plt.ylabel('Conteo')
    plt.title('Mejor cromosoma vs baseline y bandas')
    plt.legend()
    plt.grid(True, alpha=0.2)
    total = int(crom.sum())
    plt.text(0.02, 0.95, f'Suma={total} (objetivo {SUMA_OBJ})', transform=plt.gca().transAxes)
    out = FIGS / 'cromosoma_barras.png'
    plt.savefig(out, dpi=150, bbox_inches='tight')
    plt.show()
    print('Guardado:', out)
else:
    print('Falta archivo:', P_MEJOR_CROM)

## Insight 4: Layout espacial por especie
Dispersión hexagonal coloreada por especie. Fuentes: `datos/grafo_hexagonal.json` + `output/mejor_asignacion_hormigas.npy`.

In [ ]:
if exists(P_GRAFO) and exists(P_ASIG):
    with open(P_GRAFO, 'r') as fh:
        grafo = json.load(fh)
    nodes = grafo['nodes']
    coords = np.array([[n['x'], n['y']] for n in nodes], dtype=float)
    asig = np.load(P_ASIG)  # one-hot (n_nodos, n_tipos)
    tipos = np.argmax(asig, axis=1)

    plt.figure(figsize=(7,6))
    cmap = ListedColormap(PALETTE)
    sc = plt.scatter(coords[:,0], coords[:,1], c=tipos, s=10, cmap=cmap)
    plt.gca().set_aspect('equal')
    plt.title('Layout hexagonal por especie')
    plt.xlabel('x')
    plt.ylabel('y')
    cbar = plt.colorbar(sc, ticks=range(asig.shape[1]))
    cbar.ax.set_yticklabels([f'E{i+1}' for i in range(asig.shape[1])])
    out = FIGS / 'layout_especies.png'
    plt.savefig(out, dpi=150, bbox_inches='tight')
    plt.show()
    print('Guardado:', out)
else:
    print('Faltan archivos:', P_GRAFO, 'o', P_ASIG)

## Insight 5: Calor de competencia por nodo
Suma de competencia con vecinos para cada nodo. Fuentes: `matriz_competencia.npy`, `grafo_hexagonal.json`, `mejor_asignacion_hormigas.npy`.

In [ ]:
if exists(P_GRAFO) and exists(P_ASIG) and exists(P_COMP):
    with open(P_GRAFO, 'r') as fh:
        grafo = json.load(fh)
    edges = np.array(grafo['edges'], dtype=int)
    nodes = grafo['nodes']
    coords = np.array([[n['x'], n['y']] for n in nodes], dtype=float)
    comp = np.load(P_COMP)
    asig = np.load(P_ASIG)
    tipos = np.argmax(asig, axis=1)
    n = len(tipos)

    calor = np.zeros(n, dtype=float)
    for i, j in edges:
        ti, tj = tipos[i], tipos[j]
        w = comp[ti, tj]
        calor[i] += w
        calor[j] += w

    plt.figure(figsize=(7,6))
    sc = plt.scatter(coords[:,0], coords[:,1], c=calor, s=12, cmap='inferno')
    plt.gca().set_aspect('equal')
    plt.title('Calor de competencia por nodo')
    plt.xlabel('x')
    plt.ylabel('y')
    cbar = plt.colorbar(sc)
    cbar.set_label('Competencia acumulada')
    out = FIGS / 'layout_calor_competencia.png'
    plt.savefig(out, dpi=150, bbox_inches='tight')
    plt.show()
    print('Guardado:', out)
else:
    print('Faltan archivos para este insight.')

## Insight 6: Matrices especie–especie (adyacencias y competencia)
Cuenta de bordes por par de especies y su ponderación por matriz de competencia.

In [ ]:
if exists(P_GRAFO) and exists(P_ASIG) and exists(P_COMP):
    with open(P_GRAFO, 'r') as fh:
        grafo = json.load(fh)
    edges = np.array(grafo['edges'], dtype=int)
    asig = np.load(P_ASIG)
    tipos = np.argmax(asig, axis=1)
    k = asig.shape[1]
    comp = np.load(P_COMP)

    # Matriz de adyacencias por especie
    M_cnt = np.zeros((k, k), dtype=int)
    # Matriz de competencia ponderada
    M_w = np.zeros((k, k), dtype=float)

    for i, j in edges:
        ti, tj = tipos[i], tipos[j]
        M_cnt[ti, tj] += 1
        M_cnt[tj, ti] += 1
        w = comp[ti, tj]
        M_w[ti, tj] += w
        M_w[tj, ti] += w

    fig, axes = plt.subplots(1, 2, figsize=(10,4.2))
    im0 = axes[0].imshow(M_cnt, cmap='Blues')
    axes[0].set_title('Adyacencias (conteo)')
    axes[0].set_xlabel('Especie j')
    axes[0].set_ylabel('Especie i')
    axes[0].set_xticks(range(k), [f'E{x+1}' for x in range(k)], rotation=45)
    axes[0].set_yticks(range(k), [f'E{x+1}' for x in range(k)])
    fig.colorbar(im0, ax=axes[0], fraction=0.046, pad=0.04)

    im1 = axes[1].imshow(M_w, cmap='Oranges')
    axes[1].set_title('Competencia ponderada')
    axes[1].set_xlabel('Especie j')
    axes[1].set_ylabel('Especie i')
    axes[1].set_xticks(range(k), [f'E{x+1}' for x in range(k)], rotation=45)
    axes[1].set_yticks(range(k), [f'E{x+1}' for x in range(k)])
    fig.colorbar(im1, ax=axes[1], fraction=0.046, pad=0.04)

    fig.tight_layout()
    out = FIGS / 'matrices_especies.png'
    fig.savefig(out, dpi=150, bbox_inches='tight')
    plt.show()
    print('Guardado:', out)
else:
    print('Faltan archivos para este insight.')

## Insight 7: Resumen de ejecución
Muestra el contenido de `output/resumen_ga.json` y valida archivos clave.

In [ ]:
if exists(P_RESUMEN):
    with open(P_RESUMEN, 'r', encoding='utf-8') as fh:
        resumen = json.load(fh)
    display(pd.DataFrame([resumen]))
    out = FIGS / 'resumen_ga.json'
    with open(out, 'w', encoding='utf-8') as fh:
        json.dump(resumen, fh, ensure_ascii=False, indent=2)
    print('Copia guardada en:', out)
else:
    print('Falta archivo:', P_RESUMEN)

print('Archivos presentes:')
for p in [P_MEJOR_CROM, P_HIST_GA, P_RESUMEN, P_ASIG, P_HIST_ACO]:
    print('-', p.name, '->', 'OK' if exists(p) else 'NO')